In [1]:
pip install praw

Note: you may need to restart the kernel to use updated packages.


In [1]:
import praw

In [2]:
reddit = praw.Reddit(client_id='fYRWm5jkNOpIWA', client_secret='zKi7IdnZZ-5xDxw_jc-HBZpEjN0HZQ', user_agent='RedBat')

In [19]:
hot_posts = reddit.subreddit('wallstreetbets').new(limit=10)

for post in hot_posts:
    print(post.url)
    print(post.title)
    print(post.selftext)

https://www.reddit.com/r/wallstreetbets/comments/l8u7ay/if_i_had_the_resources_to_do_it_i_would_get_a/
If I had the resources to do it I would Get a media ad for $GME 💎💎✋✋💎💎
This is purely my opinion and my thoughts if I had a few million sitting around:

If Melvin capital has the right to post misinformation to cause panic and try to influence a market price why can’t I do the same? Billboards are great, but think out of the box here! How is it legal to actually put an ad on a news site and spread misinformation like Melvin is currently doing? They are trying to panic retail investors into selling. Only problem is CNBC now only appeals to big money and conservative boomers. This is bigger than GME, hedge funds causing massive shorts for profits should be illegal. We’re only playing under the same rules they are. They are the degens for doing this for years, not regular Joe and Mary Jane who invest in stocks. So yes I would put a commercial on prime time news to hold GME. With 💎💎✋✋💎💎. 